# LSTM Code Tutorial - Part 2

**This notebook illulstrates a tutorial on lstm using keras as sourced from keras's original documentation ([keras](https://keras.io))**

## Setup

First we will import all modules that will be used in this tutorial

In [0]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM



## Image classification using LSTM

In [0]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128


In [0]:
# The data, split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

In [0]:
# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

The architecture of our Model looks like the following:
![alt text](https://raw.githubusercontent.com/adelacru33/Deeplearning-Workshop/master/lstm/figures/LSTM-MNIST.png?raw=True)

The time distrubuted wrapper applies an LSTM to each row of the input producing 28 vectors of 128 dimensions (28x128 matrix). The time distributed wrapper is followed-up by another LSTM that outputs a 128 dimension vector. The vector then goes to a fully-connected softmax classification layer, outputing a vector of size 10.



In [0]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



In [0]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])